<a href="https://colab.research.google.com/github/julienbonin/MachineLearningApplications/blob/master/Chapters/Chapter%205/Chapter5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Problem 1:
###What is the fundamental idea behind Support Vector Machines?

#Answer:
###The fundamental idea of SVMs is to separate classes of data in a way that maximizes the distance between the two closest instances of the two classes. 


#Problem 2:

###What is a support vector?

#Answer:

###A suport vector is a datapoint(s) that is used to determine a decision boundary. From the explanaion to problem 1, SVMs want to maximize the distance between the two closest instances of the two classes. These two instances are the support vectors.

#Problem 3:

###Why is it important to scale the inputs when using SVMs?

#Answer:

###SVMs are sensitive to feature scales. If the scaling of one feature is much larger than another, the SVM may not find the largest possible space between the classes.

####aside: This is evident from looking at Figure 5-2, however, I'm still not convinced of the reasoning behind this. For example, I don't understand why the unscaled features on the left of the figure show a nearly horizontal line. If the goal of SVM is to maximize the distance between the two closest instances of two classes, why wouldn't the unscalled data work just as the scaled data? 

#Problem 4:

###Can an SVM classifier output a confidence score when it classifies an instance? What about a probability?

#Answer:

###The SVM classifier can tell you if a new data instance falls somewhere in the middle of the 'road' or 'off road' which could be used to determing some type of confidence about the classification of the new datapoint. The svm cannot tell you a probability since it only determinse whether or not a new datapoint is on one side of the street or the other. It's decision boundary is either yes or no, instead of having a probability or one class vs another.

#Problem 5:

###Should you use the primal or the dual form of the SVM problem to train a model on a training set with millions of instances and hundreds of features?

#Answer:

###For this example, you should use the primal form of the SVM problem to train the model, since the dual probelm is only faster to solve when the number of training instances are smaller than the number of features (which is not the case here).

####aside: The book seems to gloss over the details of primal and dual problems. I looked up some additional information and it seems like these concept are related to linear programming (or at least there's some representation of these ideas in LP).

#Problem 6:

###Say you trained an SVM classifier with an RBF kernel. It seems to underfit the training set: should you increase or decrease γ (gamma)? What about C?

#Answer:

###From the equation for the RBF kernel, it seems like increasing the gamma will decrease K(a,b) and decreasing gamma will increase K(a,b). Increasing K should make our decision boundary wider (similar to increasing C). Since the model is underfitting the data, we should increase gamma and/or C in order to separate the instances of each class more than they are.

#Problem 7:

###How should you set the QP parameters (H, f, A, and b) to solve the soft margin linear SVM classifier problem using an off-the-shelf QP solver?

#Answer:

###The difference between the hard and soft margin SVM classifiers is the additional C[SUM_1toM(ζ i)]. So if the number of parameters for the hard margin problem is np = n+1, then the soft margin problem with have n'p = n+1+m parameters.

###Therefore, 
###H is a (n+m+1)x(n+m+1) matrix,
###f is (n+m+1)x1 vector
###A is the same as for hard margin SVMs
###b is the same as for hard margin SVMs 


#Problem 8:

###Train a LinearSVC on a linearly separable dataset. Then train an SVC and a SGDClassifier on the same dataset. See if you can get them to produce roughly the same model.

####note: The code below is directly from the solutions repo. For this problem (and problem 9), I felt it necessary to copy from the repo. I feel like the concepts related to SVMs aren't clicking for me. This isn't the first time I've learned about SVMs. About a year ago, I took an edx course on ML, and SVMs where part of the curriculum. I understand the big picture, but I think I may be getting bogged down in the details of how this method works because some of the concepts (like Kernelization) isn't intuitive to me.

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]  # petal length, petal width
y = iris["target"]

setosa_or_versicolor = (y == 0) | (y == 1)
X = X[setosa_or_versicolor]
y = y[setosa_or_versicolor]

In [ ]:
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

C = 5
alpha = 1 / (C * len(X))

lin_clf = LinearSVC(loss="hinge", C=C, random_state=42)
svm_clf = SVC(kernel="linear", C=C)
sgd_clf = SGDClassifier(loss="hinge", learning_rate="constant", eta0=0.001, alpha=alpha,
                        max_iter=1000, tol=1e-3, random_state=42)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

lin_clf.fit(X_scaled, y)
svm_clf.fit(X_scaled, y)
sgd_clf.fit(X_scaled, y)

print("LinearSVC:                   ", lin_clf.intercept_, lin_clf.coef_)
print("SVC:                         ", svm_clf.intercept_, svm_clf.coef_)
print("SGDClassifier(alpha={:.5f}):".format(sgd_clf.alpha), sgd_clf.intercept_, sgd_clf.coef_)

LinearSVC:                    [0.28474272] [[1.05364736 1.09903308]]
SVC:                          [0.31896852] [[1.1203284  1.02625193]]
SGDClassifier(alpha=0.00200): [0.117] [[0.77714169 0.72981762]]


In [ ]:
# Compute the slope and bias of each decision boundary
w1 = -lin_clf.coef_[0, 0]/lin_clf.coef_[0, 1]
b1 = -lin_clf.intercept_[0]/lin_clf.coef_[0, 1]
w2 = -svm_clf.coef_[0, 0]/svm_clf.coef_[0, 1]
b2 = -svm_clf.intercept_[0]/svm_clf.coef_[0, 1]
w3 = -sgd_clf.coef_[0, 0]/sgd_clf.coef_[0, 1]
b3 = -sgd_clf.intercept_[0]/sgd_clf.coef_[0, 1]

# Transform the decision boundary lines back to the original scale
line1 = scaler.inverse_transform([[-10, -10 * w1 + b1], [10, 10 * w1 + b1]])
line2 = scaler.inverse_transform([[-10, -10 * w2 + b2], [10, 10 * w2 + b2]])
line3 = scaler.inverse_transform([[-10, -10 * w3 + b3], [10, 10 * w3 + b3]])

# Plot all three decision boundaries
plt.figure(figsize=(11, 4))
plt.plot(line1[:, 0], line1[:, 1], "k:", label="LinearSVC")
plt.plot(line2[:, 0], line2[:, 1], "b--", linewidth=2, label="SVC")
plt.plot(line3[:, 0], line3[:, 1], "r-", label="SGDClassifier")
plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs") # label="Iris versicolor"
plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo") # label="Iris setosa"
plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.legend(loc="upper center", fontsize=14)
plt.axis([0, 5.5, 0, 2])

plt.show()

#Problem 9

###Train an SVM classifier on the MNIST dataset. Since SVM classifiers are binary classifiers, you will need to use one-versus-all to classify all 10 digits. You may want to tune the hyperparameters using small validation sets to speed up the process. What accuracy can you reach?

####Note: The code below is directly from the solutions repo. I've added annotations to explain what is being performed, along with any lack of understanding I may (or may not) have.

In [ ]:
# This cell is just as before with the mnist dataset. The first 60k is the training data, the rest is the test data
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True)

X = mnist["data"]
y = mnist["target"].astype(np.uint8)

X_train = X[:60000]
y_train = y[:60000]
X_test = X[60000:]
y_test = y[60000:]

In [ ]:
# This cell creates and fits a Linear SVC model to the training data
lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train, y_train)

In [ ]:
# This cell calculates the accuracy of the model to the training data. 
# The repo (after running this cell) mentions that the 89.5% accuracy
# is pretty bad. I'm a little confused about why this would be bad, 
# since we don't want to overfit on the training data. 95-100%
# accuracy definitely seems like overfitting the training set, so 
# I'm not sure what I'm missing here.
from sklearn.metrics import accuracy_score

y_pred = lin_clf.predict(X_train)
accuracy_score(y_train, y_pred)

In [ ]:
# This cell scales the data. This alludes back to the about problem (#3) where
# I mention about the purpose of scalling the data. Clearly from running these
# cells, you can see that the SVC classifier performs better if the data is 
# scaled, however, I don't think I fully understand why this is the case.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled = scaler.transform(X_test.astype(np.float32))

In [ ]:
# This cell does the same as the previous few cells, except with the scaled data.
lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train_scaled, y_train)

y_pred = lin_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

svm_clf = SVC(gamma="scale")
svm_clf.fit(X_train_scaled[:10000], y_train[:10000])

y_pred = svm_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

In [ ]:
# This cell contains the remainder of the code from the solutions repo.
# Here, the hyper parameters and best model are selected and tested.
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(svm_clf, param_distributions, n_iter=10, verbose=2, cv=3)
rnd_search_cv.fit(X_train_scaled[:1000], y_train[:1000])

rnd_search_cv.best_estimator_

rnd_search_cv.best_score_

rnd_search_cv.best_estimator_.fit(X_train_scaled, y_train)

y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

#Problem 10:

###Train an SVM regressor on the California housing dataset.

####note: This problem seems straigt forward, so I did much of the implementation myself. I used the solution repo as a guide. Anything that was taken from the solution is labeled at the top of the cell along with any comments.

In [ ]:
# This was copied from the repo. It's essentially the same as in chapter 2
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
X = housing["data"]
y = housing["target"]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Now that we have the data (X,y)=(data, labels), and the test and training set, we can work on 
# selecting a model. Similar to problem 9, I wanted to try to create a model without scaling the data first.

from sklearn.svm import LinearSVR

lin_svr = LinearSVR(random_state=42)
lin_svr.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=42, tol=0.0001, verbose=0)

In [ ]:
# After fitting the training data, lets see how it performs
# In the housing example, we used MSE
from sklearn.metrics import mean_squared_error
import numpy as np

y_pred = lin_svr.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
print('MSE: ', mse)
print('RMSE: ', np.sqrt(mse))


MSE:  1.8317654224357183
RMSE:  1.3534272874579256


In [ ]:
# Comparing these results from the solutions repo, it appears that the error is much larger on
# the unscaled data. Below is the code from the solution repo for the scaled data error.



from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lin_svr = LinearSVR(random_state=42)
lin_svr.fit(X_train_scaled, y_train)

y_pred = lin_svr.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
print('MSE: ', mse)
print('RMSE: ', np.sqrt(mse))

MSE:  5571.747899808996
RMSE:  74.64414176483642


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


####note: I'm not sure what my error here is. For the scaled data, the MSE and RMSE seem to be very far from what I expected. I will review the code and continue from here after I fix the issue.  